# Import Section
---

In [1]:
import regex as re
import shutil
import os
from subprocess import Popen
import subprocess


import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from ipywidgets import AppLayout, Button, Layout, Box, FloatText, Textarea, Dropdown, Label, IntSlider
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

In [2]:
class train_config_and_cmds_widgets():
    def __init__(self):
        
        self.tflite_file_loc = ""
        self.datainfo_pt = None
        self.cfg_pt = None
        self.ori_weight_pt = None
        self.gen_pretrain_pt = None
        self.darknet_pt = os.path.join(os.getcwd(), 'Yolo-Fastest-darknet', 'build', 'darknet', 'x64', 'darknet.exe')
        
        form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
        )
        
        ### create proj ###
        self.A_cp = widgets.Text(value='MyTask', placeholder='Type something', disabled=False)
        self.B_cp = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.C_cp = widgets.BoundedIntText(value=1, min=1, max=1000, step=1, disabled=False)
        self.D_cp = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.E_cp = widgets.Text(value='..', placeholder='Type something', disabled=False)
        
        form_proj_items = [
            Box([Label(value = 'Project Name'),                          self.A_cp], layout=form_item_layout),
            Box([Label(value = 'Dataset Classes'),                 self.C_cp], layout=form_item_layout),
            Box([Label(value = 'Choose the Dataset Dir'),                 self.D_cp], layout=form_item_layout),
            Box([Label(value = 'The Chosen Dataset'),                 self.E_cp], layout=form_item_layout),
            Box([Label(value = 'Create Project Folder'),                 self.B_cp], layout=form_item_layout),
        ]
        
        self.form_create_proj= Box(form_proj_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))
        
        ### train ###
        self.A_ta = Dropdown(options=[320])
        self.B_ta = Dropdown(options=[320])
        self.C_ta = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        
        form_train_items = [
            Box([Label(value = 'width'),                          self.A_ta], layout=form_item_layout),
            Box([Label(value = 'height'),                         self.B_ta], layout=form_item_layout),
            Box([Label(value = 'Prepare the training'),           self.C_ta], layout=form_item_layout),
        ]
        
        self.form_output_train_cmd = Box(form_train_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))
        
        ### test ###
        self.A_tt = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.B_tt = widgets.BoundedIntText(value=1,min=1,max=100,step=1,disabled=False)
        self.C_tt = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
      
        form_test_items = [
            Box([Label(value = 'Choose the tflite file'), self.A_tt], layout=form_item_layout),
            Box([Label(value = 'Batches for test'),       self.B_tt], layout=form_item_layout),
            Box([Label(value = 'Start to Test'),          self.C_tt], layout=form_item_layout),
        ]
        
        self.form_output_test_cmd = Box(form_test_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        ### convert model cpp ###
        self.A_cm = widgets.Text(value='..\workspace\\catsdogs\\tflite_model', placeholder='Type something', disabled=False)
        self.B_cm = widgets.Text(value='mobilenet_v2_int8quant.tflite', placeholder='Type something', disabled=False)
        self.C_cm = widgets.Text(value='..\workspace\\catsdogs\\tflite_model\\vela', placeholder='Type something', disabled=False)
        self.E_cm = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.D_cm = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.F_cm = widgets.Text(value='cats_and_dogs_filtered', placeholder='Type something', disabled=False)
        self.G_cm = widgets.Checkbox(value=True, disabled=False, indent=False)
      
        form_convert_items_paths =  [
            Box([Label(value = 'Choose the tflite file'), self.E_cm], layout=form_item_layout),
            Box([Label(value = 'MODEL SRC DIR'),          self.A_cm], layout=form_item_layout),
            Box([Label(value = 'MODEL SRC FILE'),         self.B_cm], layout=form_item_layout),
            Box([Label(value = 'GEN SRC DIR'),            self.C_cm], layout=form_item_layout)
        ]
        
        form_convert_items_label =  [
           Box([Label(value = 'Dataset Name'), self.F_cm], layout=form_item_layout),
            Box([Label(value = 'Labels Including'), self.G_cm], layout=form_item_layout),
        ]
        
        form_convert_items = [
            Box(form_convert_items_paths, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',    
            border='dotted 3px lightblue',
            align_items='stretch',
            width='70%')),
            Box(form_convert_items_label, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',
            border='dotted 3px lightblue',
            align_items='stretch',
            width='70%')),
            Box([Label(value = 'Convert to cpp & Vela'), self.D_cm], layout=form_item_layout),
        ]
        
        self.form_output_convert_cmd = Box(form_convert_items, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='70%',
        ))
        
    def move_allfiles(self, src_folder, dst_folder):
        copy_num = 0
        
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  #copy whole folder
                shutil.move(fullpath, dst_folder)
                print("Copy finish: {}".format(f))
    
    def show_headline(self, output):
        html0= widgets.HTML(value = f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)
    
    def show_main(self):   
        
        intro_text = 'Please Choose the setting of data prepare & train'
        htmlWidget = widgets.HTML(value = f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(htmlWidget)
        
        #Create the accordion
        accordion = widgets.Accordion(children=[self.form_create_proj, self.form_output_train_cmd
                                                ]).add_class("parentstyle")
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, 'Create the Project')
        accordion.set_title(1, 'Configure the Training')
        
        # Create a box combining with 2 elements
        box_data_train = Box([accordion], layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        #Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[box_data_train, self.form_output_test_cmd, self.form_output_convert_cmd]).add_class("parentstyle")
        tab_contents = ['Train', 'Test', 'Deployment']
        tab.titles = tab_contents
       
        
        #Add a custom style tag to the notebook, you can use dev tool to inspect the class names
        #display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        #accordion.set_title(0, 'Configure the Training')        
        output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))

        def act_para(
                     A_cm, B_cm, C_cm, F_cm, G_cm, B_tt, 
                     ):
        
            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()
        
        #------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        #------------------#
        out_inter = widgets.interactive_output(act_para, { 
                                                          'A_cm': self.A_cm, 'B_cm': self.B_cm, 'C_cm': self.C_cm,
                                                          'F_cm': self.F_cm, 'G_cm': self.G_cm, 'B_tt': self.B_tt,
                                                          })

        display(tab, out_inter)
        
        #------------------#
        # for labelimg cmd, move to outside of act_para to prevent keep trigering
        #------------------#
        #output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        display(output_widgets)
        
        def on_button_clicked_createproj(b):
                with output_widgets:
                    clear_output()
                    print("Create Project Folders. . .")
                    self.create_dirs()       
        self.B_cp.on_click(on_button_clicked_createproj)
        
        def on_button_clicked_choose_dataset_dir(b):
                with output_widgets:
                    clear_output()
                    self.choose_dataset_dir()       
        self.D_cp.on_click(on_button_clicked_choose_dataset_dir)
        
        def on_button_clicked_prepare_train(b):
                with output_widgets:
                    clear_output()
                    self.prepare_train()       
        self.C_ta.on_click(on_button_clicked_prepare_train)
        
        def on_button_clicked_cpp(b):
                with output_widgets:
                    clear_output()
                    print("Convert to cpp & Vela. . .")
                    self.convert_tflu()       
        self.D_cm.on_click(on_button_clicked_cpp)
        
        def on_button_clicked_choose_tflite_test(b):
                with output_widgets:
                    clear_output()
                    self.choose_tflite()       
        self.A_tt.on_click(on_button_clicked_choose_tflite_test)
        
        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.run_test_tflite()           
        self.C_tt.on_click(on_button_clicked_test)

    def choose_dataset_dir(self):
        
        path_ftflite = os.path.join(os.getcwd())
        f_dataset = FileChooser(path_ftflite)
        f_dataset.show_only_dirs = True
        # Restrict navigation to /Users
        #f_dataset.sandbox_path = os.getcwd()
        # f_dataset.filter_pattern = ['*.tflite']
        f_dataset.title = f"<b><font color='lightblue'><font size=4>Choose the directory of dataset.</b>"
        display(f_dataset)
        
        def act_test():
            m_src_dir = f_dataset.selected_path
            self.E_cp.value = m_src_dir
            print("The chosen dir: {}".format(self.E_cp.value))
            print("Finish!")
        evt = interact_manual(act_test)
        evt.widget.children[0].description = 'Choose this dataset'  #because there are 3 parameter of the evt
        evt.widget.children[0].button_style = 'primary'
        
    def create_dirs(self):
        proj_path = os.path.join(os.getcwd(), 'workspace', self.A_cp.value)
        source_path_mzoo = os.path.join(os.getcwd(), 'Yolo-Fastest-darknet','ModelZoo','yolo-fastest-1.1_coco')
        source_path_dataset = self.E_cp.value
        if not os.path.exists(proj_path):
            os.makedirs(proj_path)
            os.makedirs(os.path.join(proj_path,'backup'))
            # copy from ModelZoo
            shutil.copy(os.path.join(source_path_mzoo, 'yolo-fastest-1.1.cfg'), proj_path)
            shutil.copy(os.path.join(source_path_mzoo, 'coco.data'), proj_path)
            # copy from user's dataset
            coconame_src_pth = os.path.join(source_path_dataset, 'coco.names')
            traintxt_src_pth = os.path.join(source_path_dataset, 'train.txt')
            valtxt_src_pth   = os.path.join(source_path_dataset, 'val.txt')
            if os.path.exists(coconame_src_pth) and os.path.exists(traintxt_src_pth) and os.path.exists(valtxt_src_pth):
                shutil.copy(coconame_src_pth, proj_path)
                # update coco.data
                file1 = open(os.path.join(proj_path, 'coco.data'), 'r')
                lines_list = file1.readlines() 
                lines_list[0] = 'classes= {}\n'.format(self.C_cp.value)
                lines_list[1] = 'train  = {}\n'.format(traintxt_src_pth)
                lines_list[2] = 'valid  = {}\n'.format(valtxt_src_pth)
                lines_list[3] = 'names = {}\n'.format(os.path.join(proj_path, 'coco.names'))
                lines_list[4] = 'backup = {}\n'.format(os.path.join('workspace', self.A_cp.value, 'backup'))
                file1.close()
                file1 = open(os.path.join(proj_path, 'coco.data'), 'w')
                file1.writelines(lines_list)
                file1.close()
                print("Finish!")
            else:
                print("There are files missing in dataset! Please check: {}".format(source_path_dataset))
        else:
            print("This project is exist! : {}".format(proj_path)) 

        return proj_path
    
    def prepare_train(self):
        proj_path = os.path.join(os.getcwd(), 'workspace', self.A_cp.value)
        self.datainfo_pt = '.\workspace\{}\coco.data'.format(self.A_cp.value)
        self.cfg_pt = '.\workspace\{}\yolo-fastest-1.1.cfg'.format(self.A_cp.value)
        self.ori_weight_pt = '.\Yolo-Fastest-darknet\ModelZoo\yolo-fastest-1.1_coco\yolo-fastest-1.1.weights'
        self.gen_pretrain_pt = '.\workspace\{}\yolo-fastest-1.1.conv.109'.format(self.A_cp.value)
        
        # run calc_anchors
        print('Start to run calc_anchors, please waiting for couple of minutes!')
        CMD_anchors =[self.darknet_pt, 'detector', 'calc_anchors', self.datainfo_pt, '-num_of_clusters', '6', '-width', str(self.A_ta.value), '-height', str(self.B_ta.value)]
        p = Popen(CMD_anchors)
        stdout, stderr = p.communicate()
        if stderr:
            print(stderr)
        else:
            print('calc_anchors done!')
            
        # update cfg
        file_anchor = open('anchors.txt', 'r')
        anchor_str = file_anchor.readlines()[0].split('\n')[0] 
        file_anchor.close()
        
        file1 = open(os.path.join(proj_path, 'yolo-fastest-1.1.cfg'), 'r')
        lines_list = file1.readlines()
        lines_list[3] = 'width={}\n'.format(self.A_ta.value)
        lines_list[4] = 'height={}\n'.format(self.B_ta.value)
        lines_list[863] = 'anchors ={}\n'.format(anchor_str)
        lines_list[931] = 'anchors ={}\n'.format(anchor_str)
        lines_list[864] = 'classes={}\n'.format(self.C_cp.value)
        lines_list[932] = 'classes={}\n'.format(self.C_cp.value)
        lines_list[857] = 'filters={}\n'.format((self.C_cp.value+5)*3)
        lines_list[925] = 'filters={}\n'.format((self.C_cp.value+5)*3)
        file1.close()
        
        file1 = open(os.path.join(proj_path, 'yolo-fastest-1.1.cfg'), 'w')
        file1.writelines(lines_list)
        file1.close()
        print('update training config done!')
        
        # get the pre-train backbone
        CMD_partial = [self.darknet_pt, 'partial', self.cfg_pt, self.ori_weight_pt, self.gen_pretrain_pt, '109']
        p = Popen(CMD_partial)
        stdout, stderr = p.communicate()
        if stderr:
            print(stderr)
        else:
            print('partial done!')

        # get the training cmd
        self.show_headline('Please copy these commands to the CMD.exe Prompt to execute. ')
        
        self.show_headline('(1.) Please excute the below commands under the working directory')
        print("cd {}".format(os.path.join(os.getcwd())))
        self.show_headline('(2.) Train:')
        train_cmd = r".\Yolo-Fastest-darknet\build\darknet\x64\darknet.exe detector train {} {} {}".format(self.datainfo_pt, self.cfg_pt, self.gen_pretrain_pt)
        print(train_cmd)
            
    def convert_tflu(self):
        
        %run exebat.py --SRC_DIR $self.A_cm.value --SRC_FILE $self.B_cm.value --GEN_DIR $self.C_cm.value
        
        if self.G_cm.value: # create the label C++ source/header files
            # Change to dataset folder
            old_cwd = os.getcwd()
            batch_cwd = os.path.join(old_cwd, "dataset")
            os.chdir(batch_cwd)
            
            %run gen_labels_cpp.py --labels_dataset_name $self.F_cm.value --source_folder_path $self.A_cm.value \
            --header_folder_path $self.A_cm.value
            
            os.chdir(old_cwd)
            print('Finish, the label file is at: {}'.format((old_cwd + self.A_cm.value.split('..')[1])))
            

In [3]:
act = train_config_and_cmds_widgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of data prepare & train</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…